In [ ]:
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.mask import mask
from shapely.geometry import box
import rasterio.transform
from rasterio.features import geometry_mask
import matplotlib.pyplot as plt

# Read the region polygon shapefile
region = gpd.read_file('DIRECTORY_PATH/AmazonBiome_boundaries.shp')

# Read the deforestation raster
deforestation_raster = rasterio.open('DIRECTORY_PATH/Deforested_areas.tif')

# Read the additional deforestation raster
deforestation_suitable_raster = rasterio.open('DIRECTORY_PATH/SoySuitable_Deforested_areas.tif')

# Clip the rasters to the extent of the region polygon
bbox = region.geometry.total_bounds
clipped_raster, clipped_transform = mask(deforestation_raster, [box(*bbox)], crop=True)
clipped_suitable_raster, _ = mask(deforestation_suitable_raster, [box(*bbox)], crop=True)

# Extract the deforested areas (pixels with value 1)
deforestation_array = clipped_raster[0]
deforested_pixels = np.where(deforestation_array == 1)

# Extract the deforestation suitable areas (pixels with value 1)
deforestation_suitable_array = clipped_suitable_raster[0]
deforestation_suitable_pixels = np.where(deforestation_suitable_array == 1)

# Convert pixel coordinates to CRS of the region polygon
deforested_coords = rasterio.transform.xy(clipped_transform, deforested_pixels[0], deforested_pixels[1])
deforestation_suitable_coords = rasterio.transform.xy(clipped_transform, deforestation_suitable_pixels[0], deforestation_suitable_pixels[1])

# Read the forest raster
forest_raster = rasterio.open('DIRECTORY_PATH/ForestCover2020.tif')

# Create a mask for the forest raster based on the geometry of the region polygon
forest_mask = geometry_mask(region.geometry, out_shape=(forest_raster.height, forest_raster.width), transform=forest_raster.transform, invert=True)

# Apply the mask to extract forest pixels that intersect with the region polygon
forest_pixels = np.where((forest_raster.read(1) == 1) & forest_mask)

# Convert pixel coordinates to CRS of the region polygon
forest_coords = rasterio.transform.xy(forest_raster.transform, forest_pixels[0], forest_pixels[1])

# Plot the region polygon with custom color and no edge color
fig, ax = plt.subplots(figsize=(8, 8))
region.plot(ax=ax, color='white', edgecolor='black', zorder=1)

# Plot the deforested areas as gold dots without edges
deforested_plot = plt.scatter(*deforested_coords, color='firebrick', s=0.3, alpha=0.03, zorder=3, label='Forest cover loss 2008-2020')

# Plot the deforestation suitable areas as yellow dots without edges
deforestation_suitable_plot = plt.scatter(*deforestation_suitable_coords, color='yellow', s=0.3, alpha=0.03, zorder=4, label='Soy suitable forest cover loss 2008-2020')

# Plot the forest areas as green dots without edges
forest_plot = plt.scatter(*forest_coords, color='mediumseagreen', s=0.85, alpha=0.15, zorder=2, label='Forest cover 2020', edgecolor='none')

# Read additional shapefiles
ports_shapefile = gpd.read_file('DIRECTORY_PATH/PortsAmazon.shp')
highways_shapefile = gpd.read_file('DIRECTORY_PATH/RoadsAmazon.shp')

# Reproject shapefiles to match the CRS of the region polygon
ports_shapefile = ports_shapefile.to_crs(region.crs)
highways_shapefile = highways_shapefile.to_crs(region.crs)

# Plot additional shapefiles (points and polylines)
highways_plot = highways_shapefile.plot(ax=ax, color='black', linewidth=0.5, linestyle='--', marker='o', markersize=0.1, zorder=5, label='Roads')
ports_plot = ports_shapefile.plot(ax=ax, marker='o', color='red', edgecolor='black', markersize=40, zorder=6, label='Ports', legend=True)

plt.gca().set_aspect('equal', adjustable='box')

# Create legend without the frame, and change the order of the legend labels
legend_elements = [
    plt.Rectangle((0, 0), 1, 1, fc="white", ec="black", lw=2, label="Amazon Biome"),
    plt.Line2D([0], [0], marker='s', color='w', label='Forest cover 2020', markerfacecolor='mediumseagreen', markeredgecolor='black', markersize=10, alpha=1),
    plt.Line2D([0], [0], marker='s', color='w', label='Deforested areas 2008-2020', markerfacecolor='firebrick', markeredgecolor='black', markersize=10, alpha=1),
    plt.Line2D([0], [0], marker='s', color='w', label='Soy suitable deforested areas 2008-2020', markerfacecolor='gold', markeredgecolor='black', markersize=10, alpha=1),
    plt.Line2D([0], [0], color='black', lw=2, linestyle='--', label='Roads'),
    plt.Line2D([0], [0], marker='o', color='w', label='Operational ports', markerfacecolor='red', markeredgecolor='black', markersize=10),
]

# Create legend without the frame, and change the order of the legend labels
legend = plt.legend(handles=legend_elements, loc='lower right', frameon=True)

# Set legend font size
plt.setp(legend.get_texts(), fontsize='small')  # Set legend text size

# Set legend box size
legend.get_frame().set_linewidth(0.5)  # Set legend box border width
legend.get_frame().set_edgecolor('black')  # Set legend box border color

# Adjust the position of the legend to the right
legend.set_bbox_to_anchor((1.25, 0.04))

# Remove the square frame around the entire map
ax.axis('off')

plt.show()
